<a href="https://colab.research.google.com/github/CShorten/Small-Weaviate-Examples/blob/main/Recurring-Spans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Python implementation to find <b>Recurring Spans</b> </h1>

This notebook presents the idea behind <b> Recurring Spans </b>, the strategy used to Self-Supervised anchor, positive, and negative construction in "Learning to Retrieve Passages without Supervision" - Ram et al. 2022.
<br />
This is done in 4 steps:
<ol>
<li> We begin by chunking up a <b>Document</b> into 100 word chunks called <b>Passages</b></li>
<li> Extract <b>spans / n-grams</b> from each of the <b>Passages</b></li>
<li> Count the <b>Recurring Spans</b> across <b>Passages</b></li>
<li> Sort <b>Recurring Spans</b> by length and take the longest matching passages as <b>Positives</b></li>
</ol>

In [15]:
# copy and paste Document from Wikipedia -- this is the Wikipedia article of Deep Learning
Document = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised.[2]

Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks and convolutional neural networks have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.[3][4][5]

Artificial neural networks (ANNs) were inspired by information processing and distributed communication nodes in biological systems. ANNs have various differences from biological brains. Specifically, artificial neural networks tend to be static and symbolic, while the biological brain of most living organisms is dynamic (plastic) and analogue.[6][7]

The adjective "deep" in deep learning refers to the use of multiple layers in the network. Early work showed that a linear perceptron cannot be a universal classifier, but that a network with a nonpolynomial activation function with one hidden layer of unbounded width can. Deep learning is a modern variation which is concerned with an unbounded number of layers of bounded size, which permits practical application and optimized implementation, while retaining theoretical universality under mild conditions. In deep learning the layers are also permitted to be heterogeneous and to deviate widely from biologically informed connectionist models, for the sake of efficiency, trainability and understandability, whence the "structured" part.

Definition
Deep learning is a class of machine learning algorithms that[8]: 199–200  uses multiple layers to progressively extract higher-level features from the raw input. For example, in image processing, lower layers may identify edges, while higher layers may identify the concepts relevant to a human such as digits or letters or faces.

Overview
Most modern deep learning models are based on artificial neural networks, specifically convolutional neural networks (CNN)s, although they can also include propositional formulas or latent variables organized layer-wise in deep generative models such as the nodes in deep belief networks and deep Boltzmann machines.[9]

In deep learning, each level learns to transform its input data into a slightly more abstract and composite representation. In an image recognition application, the raw input may be a matrix of pixels; the first representational layer may abstract the pixels and encode edges; the second layer may compose and encode arrangements of edges; the third layer may encode a nose and eyes; and the fourth layer may recognize that the image contains a face. Importantly, a deep learning process can learn which features to optimally place in which level on its own. This does not eliminate the need for hand-tuning; for example, varying numbers of layers and layer sizes can provide different degrees of abstraction.[10][11]

The word "deep" in "deep learning" refers to the number of layers through which the data is transformed. More precisely, deep learning systems have a substantial credit assignment path (CAP) depth. The CAP is the chain of transformations from input to output. CAPs describe potentially causal connections between input and output. For a feedforward neural network, the depth of the CAPs is that of the network and is the number of hidden layers plus one (as the output layer is also parameterized). For recurrent neural networks, in which a signal may propagate through a layer more than once, the CAP depth is potentially unlimited.[12] No universally agreed-upon threshold of depth divides shallow learning from deep learning, but most researchers agree that deep learning involves CAP depth higher than 2. CAP of depth 2 has been shown to be a universal approximator in the sense that it can emulate any function.[13] Beyond that, more layers do not add to the function approximator ability of the network. Deep models (CAP > 2) are able to extract better features than shallow models and hence, extra layers help in learning the features effectively.

Deep learning architectures can be constructed with a greedy layer-by-layer method.[14] Deep learning helps to disentangle these abstractions and pick out which features improve performance.[10]

For supervised learning tasks, deep learning methods eliminate feature engineering, by translating the data into compact intermediate representations akin to principal components, and derive layered structures that remove redundancy in representation.

Deep learning algorithms can be applied to unsupervised learning tasks. This is an important benefit because unlabeled data are more abundant than the labeled data. Examples of deep structures that can be trained in an unsupervised manner are deep belief networks.

Interpretations
Deep neural networks are generally interpreted in terms of the universal approximation theorem[16][17][18][19][20] or probabilistic inference.[8][10][12][21]

The classic universal approximation theorem concerns the capacity of feedforward neural networks with a single hidden layer of finite size to approximate continuous functions.[16][17][18][19] In 1989, the first proof was published by George Cybenko for sigmoid activation functions[16] and was generalised to feed-forward multi-layer architectures in 1991 by Kurt Hornik.[17] Recent work also showed that universal approximation also holds for non-bounded activation functions such as the rectified linear unit.[22]

The universal approximation theorem for deep neural networks concerns the capacity of networks with bounded width but the depth is allowed to grow. Lu et al.[20] proved that if the width of a deep neural network with ReLU activation is strictly larger than the input dimension, then the network can approximate any Lebesgue integrable function; If the width is smaller or equal to the input dimension, then a deep neural network is not a universal approximator.

The probabilistic interpretation[21] derives from the field of machine learning. It features inference,[8][9][10][12][15][21] as well as the optimization concepts of training and testing, related to fitting and generalization, respectively. More specifically, the probabilistic interpretation considers the activation nonlinearity as a cumulative distribution function.[21] The probabilistic interpretation led to the introduction of dropout as regularizer in neural networks. The probabilistic interpretation was introduced by researchers including Hopfield, Widrow and Narendra and popularized in surveys such as the one by Bishop.

History
Some sources point out that Frank Rosenblatt developed and explored all of the basic ingredients of the deep learning systems of today.[24] He described it in his book "Principles of Neurodynamics: Perceptrons and the Theory of Brain Mechanisms", published by Cornell Aeronautical Laboratory, Inc., Cornell University in 1962.

The first general, working learning algorithm for supervised, deep, feedforward, multilayer perceptrons was published by Alexey Ivakhnenko and Lapa in 1967.[25] A 1971 paper described a deep network with eight layers trained by the group method of data handling.[26] Other deep learning working architectures, specifically those built for computer vision, began with the Neocognitron introduced by Kunihiko Fukushima in 1980.[27]

The term Deep Learning was introduced to the machine learning community by Rina Dechter in 1986,[28] and to artificial neural networks by Igor Aizenberg and colleagues in 2000, in the context of Boolean threshold neurons.[29][30]

In 1989, Yann LeCun et al. applied the standard backpropagation algorithm, which had been around as the reverse mode of automatic differentiation since 1970,[31][32][33][34] to a deep neural network with the purpose of recognizing handwritten ZIP codes on mail. While the algorithm worked, training required 3 days.[35]

In 1994, André de Carvalho, together with Mike Fairhurst and David Bisset, published experimental results of a multi-layer boolean neural network, also known as a weightless neural network, composed of a 3-layers self-organising feature extraction neural network module (SOFT) followed by a multi-layer classification neural network module (GSN), which were independently trained. Each layer in the feature extraction module extracted features with growing complexity regarding the previous layer.[36]

In 1995, Brendan Frey demonstrated that it was possible to train (over two days) a network containing six fully connected layers and several hundred hidden units using the wake-sleep algorithm, co-developed with Peter Dayan and Hinton.[37] Many factors contribute to the slow speed, including the vanishing gradient problem analyzed in 1991 by Sepp Hochreiter.[38][39]

Since 1997, Sven Behnke extended the feed-forward hierarchical convolutional approach in the Neural Abstraction Pyramid[40] by lateral and backward connections in order to flexibly incorporate context into decisions and iteratively resolve local ambiguities.

Simpler models that use task-specific handcrafted features such as Gabor filters and support vector machines (SVMs) were a popular choice in the 1990s and 2000s, because of artificial neural network's (ANN) computational cost and a lack of understanding of how the brain wires its biological networks.

Both shallow and deep learning (e.g., recurrent nets) of ANNs have been explored for many years.[41][42][43] These methods never outperformed non-uniform internal-handcrafting Gaussian mixture model/Hidden Markov model (GMM-HMM) technology based on generative models of speech trained discriminatively.[44] Key difficulties have been analyzed, including gradient diminishing[38] and weak temporal correlation structure in neural predictive models.[45][46] Additional difficulties were the lack of training data and limited computing power.

Most speech recognition researchers moved away from neural nets to pursue generative modeling. An exception was at SRI International in the late 1990s. Funded by the US government's NSA and DARPA, SRI studied deep neural networks in speech and speaker recognition. The speaker recognition team led by Larry Heck reported significant success with deep neural networks in speech processing in the 1998 National Institute of Standards and Technology Speaker Recognition evaluation.[47] The SRI deep neural network was then deployed in the Nuance Verifier, representing the first major industrial application of deep learning.[48]

The principle of elevating "raw" features over hand-crafted optimization was first explored successfully in the architecture of deep autoencoder on the "raw" spectrogram or linear filter-bank features in the late 1990s,[48] showing its superiority over the Mel-Cepstral features that contain stages of fixed transformation from spectrograms. The raw features of speech, waveforms, later produced excellent larger-scale results.[49]

Many aspects of speech recognition were taken over by a deep learning method called long short-term memory (LSTM), a recurrent neural network published by Hochreiter and Schmidhuber in 1997.[50] LSTM RNNs avoid the vanishing gradient problem and can learn "Very Deep Learning" tasks[12] that require memories of events that happened thousands of discrete time steps before, which is important for speech. In 2003, LSTM started to become competitive with traditional speech recognizers on certain tasks.[51] Later it was combined with connectionist temporal classification (CTC)[52] in stacks of LSTM RNNs.[53] In 2015, Google's speech recognition reportedly experienced a dramatic performance jump of 49% through CTC-trained LSTM, which they made available through Google Voice Search.[54]

In 2006, publications by Geoff Hinton, Ruslan Salakhutdinov, Osindero and Teh[55][56][57] showed how a many-layered feedforward neural network could be effectively pre-trained one layer at a time, treating each layer in turn as an unsupervised restricted Boltzmann machine, then fine-tuning it using supervised backpropagation.[58] The papers referred to learning for deep belief nets.

Deep learning is part of state-of-the-art systems in various disciplines, particularly computer vision and automatic speech recognition (ASR). Results on commonly used evaluation sets such as TIMIT (ASR) and MNIST (image classification), as well as a range of large-vocabulary speech recognition tasks have steadily improved.[59][60] Convolutional neural networks (CNNs) were superseded for ASR by CTC[52] for LSTM.[50][54][61][62][63] but are more successful in computer vision.

The impact of deep learning in industry began in the early 2000s, when CNNs already processed an estimated 10% to 20% of all the checks written in the US, according to Yann LeCun.[64] Industrial applications of deep learning to large-scale speech recognition started around 2010.

The 2009 NIPS Workshop on Deep Learning for Speech Recognition was motivated by the limitations of deep generative models of speech, and the possibility that given more capable hardware and large-scale data sets that deep neural nets (DNN) might become practical. It was believed that pre-training DNNs using generative models of deep belief nets (DBN) would overcome the main difficulties of neural nets. However, it was discovered that replacing pre-training with large amounts of training data for straightforward backpropagation when using DNNs with large, context-dependent output layers produced error rates dramatically lower than then-state-of-the-art Gaussian mixture model (GMM)/Hidden Markov Model (HMM) and also than more-advanced generative model-based systems.[59] The nature of the recognition errors produced by the two types of systems was characteristically different,[65] offering technical insights into how to integrate deep learning into the existing highly efficient, run-time speech decoding system deployed by all major speech recognition systems.[8][66][67] Analysis around 2009–2010, contrasting the GMM (and other generative speech models) vs. DNN models, stimulated early industrial investment in deep learning for speech recognition,[65] eventually leading to pervasive and dominant use in that industry. That analysis was done with comparable performance (less than 1.5% in error rate) between discriminative DNNs and generative models.[59][65][68]

In 2010, researchers extended deep learning from TIMIT to large vocabulary speech recognition, by adopting large output layers of the DNN based on context-dependent HMM states constructed by decision trees.[69][70][71][66]

Advances in hardware have driven renewed interest in deep learning. In 2009, Nvidia was involved in what was called the “big bang” of deep learning, “as deep-learning neural networks were trained with Nvidia graphics processing units (GPUs).”[72] That year, Andrew Ng determined that GPUs could increase the speed of deep-learning systems by about 100 times.[73] In particular, GPUs are well-suited for the matrix/vector computations involved in machine learning.[74][75][76] GPUs speed up training algorithms by orders of magnitude, reducing running times from weeks to days.[77][78] Further, specialized hardware and algorithm optimizations can be used for efficient processing of deep learning models.[

Deep learning revolution

In 2012, a team led by George E. Dahl won the "Merck Molecular Activity Challenge" using multi-task deep neural networks to predict the biomolecular target of one drug.[80][81] In 2014, Hochreiter's group used deep learning to detect off-target and toxic effects of environmental chemicals in nutrients, household products and drugs and won the "Tox21 Data Challenge" of NIH, FDA and NCATS.[82][83][84]

Significant additional impacts in image or object recognition were felt from 2011 to 2012. Although CNNs trained by backpropagation had been around for decades, and GPU implementations of NNs for years, including CNNs, fast implementations of CNNs on GPUs were needed to progress on computer vision.[74][76][35][85][12] In 2011, this approach achieved for the first time superhuman performance in a visual pattern recognition contest. Also in 2011, it won the ICDAR Chinese handwriting contest, and in May 2012, it won the ISBI image segmentation contest.[86] Until 2011, CNNs did not play a major role at computer vision conferences, but in June 2012, a paper by Ciresan et al. at the leading conference CVPR[3] showed how max-pooling CNNs on GPU can dramatically improve many vision benchmark records. In October 2012, a similar system by Krizhevsky et al.[4] won the large-scale ImageNet competition by a significant margin over shallow machine learning methods. In November 2012, Ciresan et al.'s system also won the ICPR contest on analysis of large medical images for cancer detection, and in the following year also the MICCAI Grand Challenge on the same topic.[87] In 2013 and 2014, the error rate on the ImageNet task using deep learning was further reduced, following a similar trend in large-scale speech recognition.

Image classification was then extended to the more challenging task of generating descriptions (captions) for images, often as a combination of CNNs and LSTMs.[88][89][90]

Some researchers state that the October 2012 ImageNet victory anchored the start of a "deep learning revolution" that has transformed the AI industry.[91]

In March 2019, Yoshua Bengio, Geoffrey Hinton and Yann LeCun were awarded the Turing Award for conceptual and engineering breakthroughs that have made deep neural networks a critical component of computing.

Neural networks
Artificial neural networks
Main article: Artificial neural network
Artificial neural networks (ANNs) or connectionist systems are computing systems inspired by the biological neural networks that constitute animal brains. Such systems learn (progressively improve their ability) to do tasks by considering examples, generally without task-specific programming. For example, in image recognition, they might learn to identify images that contain cats by analyzing example images that have been manually labeled as "cat" or "no cat" and using the analytic results to identify cats in other images. They have found most use in applications difficult to express with a traditional computer algorithm using rule-based programming.

An ANN is based on a collection of connected units called artificial neurons, (analogous to biological neurons in a biological brain). Each connection (synapse) between neurons can transmit a signal to another neuron. The receiving (postsynaptic) neuron can process the signal(s) and then signal downstream neurons connected to it. Neurons may have state, generally represented by real numbers, typically between 0 and 1. Neurons and synapses may also have a weight that varies as learning proceeds, which can increase or decrease the strength of the signal that it sends downstream.

Typically, neurons are organized in layers. Different layers may perform different kinds of transformations on their inputs. Signals travel from the first (input), to the last (output) layer, possibly after traversing the layers multiple times.

The original goal of the neural network approach was to solve problems in the same way that a human brain would. Over time, attention focused on matching specific mental abilities, leading to deviations from biology such as backpropagation, or passing information in the reverse direction and adjusting the network to reflect that information.

Neural networks have been used on a variety of tasks, including computer vision, speech recognition, machine translation, social network filtering, playing board and video games and medical diagnosis.

As of 2017, neural networks typically have a few thousand to a few million units and millions of connections. Despite this number being several order of magnitude less than the number of neurons on a human brain, these networks can perform many tasks at a level beyond that of humans (e.g., recognizing faces, playing "Go"[92] ).

Deep neural networks
A deep neural network (DNN) is an artificial neural network (ANN) with multiple layers between the input and output layers.[9][12] There are different types of neural networks but they always consist of the same components: neurons, synapses, weights, biases, and functions.[93] These components functioning similar to the human brains and can be trained like any other ML algorithm.[citation needed]

For example, a DNN that is trained to recognize dog breeds will go over the given image and calculate the probability that the dog in the image is a certain breed. The user can review the results and select which probabilities the network should display (above a certain threshold, etc.) and return the proposed label. Each mathematical manipulation as such is considered a layer,[citation needed] and complex DNN have many layers, hence the name "deep" networks.

DNNs can model complex non-linear relationships. DNN architectures generate compositional models where the object is expressed as a layered composition of primitives.[94] The extra layers enable composition of features from lower layers, potentially modeling complex data with fewer units than a similarly performing shallow network.[9] For instance, it was proved that sparse multivariate polynomials are exponentially easier to approximate with DNNs than with shallow networks.[95]

Deep architectures include many variants of a few basic approaches. Each architecture has found success in specific domains. It is not always possible to compare the performance of multiple architectures, unless they have been evaluated on the same data sets.

DNNs are typically feedforward networks in which data flows from the input layer to the output layer without looping back. At first, the DNN creates a map of virtual neurons and assigns random numerical values, or "weights", to connections between them. The weights and inputs are multiplied and return an output between 0 and 1. If the network did not accurately recognize a particular pattern, an algorithm would adjust the weights.[96] That way the algorithm can make certain parameters more influential, until it determines the correct mathematical manipulation to fully process the data.

Recurrent neural networks (RNNs), in which data can flow in any direction, are used for applications such as language modeling.[97][98][99][100][101] Long short-term memory is particularly effective for this use.[50][102]

Convolutional deep neural networks (CNNs) are used in computer vision.[103] CNNs also have been applied to acoustic modeling for automatic speech recognition (ASR).

Challenges
As with ANNs, many issues can arise with naively trained DNNs. Two common issues are overfitting and computation time.

DNNs are prone to overfitting because of the added layers of abstraction, which allow them to model rare dependencies in the training data. Regularization methods such as Ivakhnenko's unit pruning[26] or weight decay ({\displaystyle \ell _{2}}\ell _{2}-regularization) or sparsity ({\displaystyle \ell _{1}}\ell _{1}-regularization) can be applied during training to combat overfitting.[105] Alternatively dropout regularization randomly omits units from the hidden layers during training. This helps to exclude rare dependencies.[106] Finally, data can be augmented via methods such as cropping and rotating such that smaller training sets can be increased in size to reduce the chances of overfitting.[107]

DNNs must consider many training parameters, such as the size (number of layers and number of units per layer), the learning rate, and initial weights. Sweeping through the parameter space for optimal parameters may not be feasible due to the cost in time and computational resources. Various tricks, such as batching (computing the gradient on several training examples at once rather than individual examples)[108] speed up computation. Large processing capabilities of many-core architectures (such as GPUs or the Intel Xeon Phi) have produced significant speedups in training, because of the suitability of such processing architectures for the matrix and vector computations.[109][110]

Alternatively, engineers may look for other types of neural networks with more straightforward and convergent training algorithms. CMAC (cerebellar model articulation controller) is one such kind of neural network. It doesn't require learning rates or randomized initial weights for CMAC. The training process can be guaranteed to converge in one step with a new batch of data, and the computational complexity of the training algorithm is linear with respect to the number of neurons involved.

Hardware
Since the 2010s, advances in both machine learning algorithms and computer hardware have led to more efficient methods for training deep neural networks that contain many layers of non-linear hidden units and a very large output layer.[113] By 2019, graphic processing units (GPUs), often with AI-specific enhancements, had displaced CPUs as the dominant method of training large-scale commercial cloud AI.[114] OpenAI estimated the hardware computation used in the largest deep learning projects from AlexNet (2012) to AlphaZero (2017), and found a 300,000-fold increase in the amount of computation required, with a doubling-time trendline of 3.4 months.[115][116]

Special electronic circuits called deep learning processors were designed to speed up deep learning algorithms. Deep learning processors include neural processing units (NPUs) in Huawei cellphones[117] and cloud computing servers such as tensor processing units (TPU) in the Google Cloud Platform.[118]

Atomically thin semiconductors are considered promising for energy-efficient deep learning hardware where the same basic device structure is used for both logic operations and data storage. In 2020, Marega et al. published experiments with a large-area active channel material for developing logic-in-memory devices and circuits based on floating-gate field-effect transistors (FGFETs).[119]

In 2021, J. Feldmann et al. proposed an integrated photonic hardware accelerator for parallel convolutional processing.[120] The authors identify two key advantages of integrated photonics over its electronic counterparts: (1) massively parallel data transfer through wavelength division multiplexing in conjunction with frequency combs, and (2) extremely high data modulation speeds.[120] Their system can execute trillions of multiply-accumulate operations per second, indicating the potential of integrated photonics in data-heavy AI applications.

Applications
Automatic speech recognition
Main article: Speech recognition
Large-scale automatic speech recognition is the first and most convincing successful case of deep learning. LSTM RNNs can learn "Very Deep Learning" tasks[12] that involve multi-second intervals containing speech events separated by thousands of discrete time steps, where one time step corresponds to about 10 ms. LSTM with forget gates[102] is competitive with traditional speech recognizers on certain tasks.[51]

The initial success in speech recognition was based on small-scale recognition tasks based on TIMIT. The data set contains 630 speakers from eight major dialects of American English, where each speaker reads 10 sentences.[121] Its small size lets many configurations be tried. More importantly, the TIMIT task concerns phone-sequence recognition, which, unlike word-sequence recognition, allows weak phone bigram language models. This lets the strength of the acoustic modeling aspects of speech recognition be more easily analyzed. The error rates listed below, including these early results and measured as percent phone error rates (PER), have been summarized since 1991.

The debut of DNNs for speaker recognition in the late 1990s and speech recognition around 2009-2011 and of LSTM around 2003–2007, accelerated progress in eight major areas:[8][68][66]

Scale-up/out and accelerated DNN training and decoding
Sequence discriminative training
Feature processing by deep models with solid understanding of the underlying mechanisms
Adaptation of DNNs and related deep models
Multi-task and transfer learning by DNNs and related deep models
CNNs and how to design them to best exploit domain knowledge of speech
RNN and its rich LSTM variants
Other types of deep models including tensor-based models and integrated deep generative/discriminative models.
All major commercial speech recognition systems (e.g., Microsoft Cortana, Xbox, Skype Translator, Amazon Alexa, Google Now, Apple Siri, Baidu and iFlyTek voice search, and a range of Nuance speech products, etc.) are based on deep learning.

Image recognition
Main article: Computer vision
A common evaluation set for image classification is the MNIST database data set. MNIST is composed of handwritten digits and includes 60,000 training examples and 10,000 test examples. As with TIMIT, its small size lets users test multiple configurations. A comprehensive list of results on this set is available.[128]

Deep learning-based image recognition has become "superhuman", producing more accurate results than human contestants. This first occurred in 2011 in recognition of traffic signs, and in 2014, with recognition of human faces.[129][130]

Deep learning-trained vehicles now interpret 360° camera views.[131] Another example is Facial Dysmorphology Novel Analysis (FDNA) used to analyze cases of human malformation connected to a large database of genetic syndromes.

Visual art processing
Closely related to the progress that has been made in image recognition is the increasing application of deep learning techniques to various visual art tasks. DNNs have proven themselves capable, for example, of

identifying the style period of a given painting[132][133]
Neural Style Transfer – capturing the style of a given artwork and applying it in a visually pleasing manner to an arbitrary photograph or video[132][133]
generating striking imagery based on random visual input fields

Natural language processing
Main article: Natural language processing
Neural networks have been used for implementing language models since the early 2000s.[97] LSTM helped to improve machine translation and language modeling.[98][99][100]

Other key techniques in this field are negative sampling[134] and word embedding. Word embedding, such as word2vec, can be thought of as a representational layer in a deep learning architecture that transforms an atomic word into a positional representation of the word relative to other words in the dataset; the position is represented as a point in a vector space. Using word embedding as an RNN input layer allows the network to parse sentences and phrases using an effective compositional vector grammar. A compositional vector grammar can be thought of as probabilistic context free grammar (PCFG) implemented by an RNN.[135] Recursive auto-encoders built atop word embeddings can assess sentence similarity and detect paraphrasing.[135] Deep neural architectures provide the best results for constituency parsing,[136] sentiment analysis,[137] information retrieval,[138][139] spoken language understanding,[140] machine translation,[98][141] contextual entity linking,[141] writing style recognition,[142] Text classification and others.[143]

Recent developments generalize word embedding to sentence embedding.

Google Translate (GT) uses a large end-to-end long short-term memory (LSTM) network.[144][145][146][147] Google Neural Machine Translation (GNMT) uses an example-based machine translation method in which the system "learns from millions of examples."[145] It translates "whole sentences at a time, rather than pieces. Google Translate supports over one hundred languages.[145] The network encodes the "semantics of the sentence rather than simply memorizing phrase-to-phrase translations".[145][148] GT uses English as an intermediate between most language pairs.

Drug discovery and toxicology
For more information, see Drug discovery and Toxicology.
A large percentage of candidate drugs fail to win regulatory approval. These failures are caused by insufficient efficacy (on-target effect), undesired interactions (off-target effects), or unanticipated toxic effects.[149][150] Research has explored use of deep learning to predict the biomolecular targets,[80][81] off-targets, and toxic effects of environmental chemicals in nutrients, household products and drugs.[82][83][84]

AtomNet is a deep learning system for structure-based rational drug design.[151] AtomNet was used to predict novel candidate biomolecules for disease targets such as the Ebola virus[152] and multiple sclerosis.[153][154]

In 2017 graph neural networks were used for the first time to predict various properties of molecules in a large toxicology data set.[155] In 2019, generative neural networks were used to produce molecules that were validated experimentally all the way into mice.

Customer relationship management
Main article: Customer relationship management
Deep reinforcement learning has been used to approximate the value of possible direct marketing actions, defined in terms of RFM variables. The estimated value function was shown to have a natural interpretation as customer lifetime value.[158]

Recommendation systems
Main article: Recommender system
Recommendation systems have used deep learning to extract meaningful features for a latent factor model for content-based music and journal recommendations.[159][160] Multi-view deep learning has been applied for learning user preferences from multiple domains.[161] The model uses a hybrid collaborative and content-based approach and enhances recommendations in multiple tasks.

Bioinformatics
Main article: Bioinformatics
An autoencoder ANN was used in bioinformatics, to predict gene ontology annotations and gene-function relationships.[162]

In medical informatics, deep learning was used to predict sleep quality based on data from wearables[163] and predictions of health complications from electronic health record data.[164]

Medical image analysis
Deep learning has been shown to produce competitive results in medical application such as cancer cell classification, lesion detection, organ segmentation and image enhancement.[165][166] Modern deep learning tools demonstrate the high accuracy of detecting various diseases and the helpfulness of their use by specialists to improve the diagnosis efficiency.[167][168]

Mobile advertising
Finding the appropriate mobile audience for mobile advertising is always challenging, since many data points must be considered and analyzed before a target segment can be created and used in ad serving by any ad server.[169] Deep learning has been used to interpret large, many-dimensioned advertising datasets. Many data points are collected during the request/serve/click internet advertising cycle. This information can form the basis of machine learning to improve ad selection.

Image restoration
Deep learning has been successfully applied to inverse problems such as denoising, super-resolution, inpainting, and film colorization.[170] These applications include learning methods such as "Shrinkage Fields for Effective Image Restoration"[171] which trains on an image dataset, and Deep Image Prior, which trains on the image that needs restoration.

Financial fraud detection
Deep learning is being successfully applied to financial fraud detection, tax evasion detection,[172] and anti-money laundering.[173]

Military
The United States Department of Defense applied deep learning to train robots in new tasks through observation.[174]

Partial differential equations
Physics informed neural networks have been used to solve partial differential equations in both forward and inverse problems in a data driven manner.[175] One example is the reconstructing fluid flow governed by the Navier-Stokes equations. Using physics informed neural networks does not require the often expensive mesh generation that conventional CFD methods relies on.[176][177]

Image Reconstruction
Image reconstruction is the reconstruction of the underlying images from the image-related measurements. Several works showed the better and superior performance of the deep learning methods compared to analytical methods for various applications, e.g., spectral imaging [178] and ultrasound imaging.[179]

Relation to human cognitive and brain development
Deep learning is closely related to a class of theories of brain development (specifically, neocortical development) proposed by cognitive neuroscientists in the early 1990s.[180][181][182][183] These developmental theories were instantiated in computational models, making them predecessors of deep learning systems. These developmental models share the property that various proposed learning dynamics in the brain (e.g., a wave of nerve growth factor) support the self-organization somewhat analogous to the neural networks utilized in deep learning models. Like the neocortex, neural networks employ a hierarchy of layered filters in which each layer considers information from a prior layer (or the operating environment), and then passes its output (and possibly the original input), to other layers. This process yields a self-organizing stack of transducers, well-tuned to their operating environment. A 1995 description stated, "...the infant's brain seems to organize itself under the influence of waves of so-called trophic-factors ... different regions of the brain become connected sequentially, with one layer of tissue maturing before another and so on until the whole brain is mature."[184]

A variety of approaches have been used to investigate the plausibility of deep learning models from a neurobiological perspective. On the one hand, several variants of the backpropagation algorithm have been proposed in order to increase its processing realism.[185][186] Other researchers have argued that unsupervised forms of deep learning, such as those based on hierarchical generative models and deep belief networks, may be closer to biological reality.[187][188] In this respect, generative neural network models have been related to neurobiological evidence about sampling-based processing in the cerebral cortex.[189]

Although a systematic comparison between the human brain organization and the neuronal encoding in deep networks has not yet been established, several analogies have been reported. For example, the computations performed by deep learning units could be similar to those of actual neurons[190] and neural populations.[191] Similarly, the representations developed by deep learning models are similar to those measured in the primate visual system[192] both at the single-unit[193] and at the population[194] levels.

Commercial activity
Facebook's AI lab performs tasks such as automatically tagging uploaded pictures with the names of the people in them.[195]

Google's DeepMind Technologies developed a system capable of learning how to play Atari video games using only pixels as data input. In 2015 they demonstrated their AlphaGo system, which learned the game of Go well enough to beat a professional Go player.[196][197][198] Google Translate uses a neural network to translate between more than 100 languages.

In 2017, Covariant.ai was launched, which focuses on integrating deep learning into factories.[199]

As of 2008,[200] researchers at The University of Texas at Austin (UT) developed a machine learning framework called Training an Agent Manually via Evaluative Reinforcement, or TAMER, which proposed new methods for robots or computer programs to learn how to perform tasks by interacting with a human instructor.[174] First developed as TAMER, a new algorithm called Deep TAMER was later introduced in 2018 during a collaboration between U.S. Army Research Laboratory (ARL) and UT researchers. Deep TAMER used deep learning to provide a robot the ability to learn new tasks through observation.[174] Using Deep TAMER, a robot learned a task with a human trainer, watching video streams or observing a human perform a task in-person. The robot later practiced the task with the help of some coaching from the trainer, who provided feedback such as “good job” and “bad job.”[201]

Criticism and comment
Deep learning has attracted both criticism and comment, in some cases from outside the field of computer science.

Theory
See also: Explainable AI
A main criticism concerns the lack of theory surrounding some methods.[202] Learning in the most common deep architectures is implemented using well-understood gradient descent. However, the theory surrounding other algorithms, such as contrastive divergence is less clear.[citation needed] (e.g., Does it converge? If so, how fast? What is it approximating?) Deep learning methods are often looked at as a black box, with most confirmations done empirically, rather than theoretically.[203]

Others point out that deep learning should be looked at as a step towards realizing strong AI, not as an all-encompassing solution. Despite the power of deep learning methods, they still lack much of the functionality needed for realizing this goal entirely. Research psychologist Gary Marcus noted:

"Realistically, deep learning is only part of the larger challenge of building intelligent machines. Such techniques lack ways of representing causal relationships (...) have no obvious ways of performing logical inferences, and they are also still a long way from integrating abstract knowledge, such as information about what objects are, what they are for, and how they are typically used. The most powerful A.I. systems, like Watson (...) use techniques like deep learning as just one element in a very complicated ensemble of techniques, ranging from the statistical technique of Bayesian inference to deductive reasoning."[204]

In further reference to the idea that artistic sensitivity might be inherent in relatively low levels of the cognitive hierarchy, a published series of graphic representations of the internal states of deep (20-30 layers) neural networks attempting to discern within essentially random data the images on which they were trained[205] demonstrate a visual appeal: the original research notice received well over 1,000 comments, and was the subject of what was for a time the most frequently accessed article on The Guardian's[206] website.

Errors
Some deep learning architectures display problematic behaviors,[207] such as confidently classifying unrecognizable images as belonging to a familiar category of ordinary images (2014)[208] and misclassifying minuscule perturbations of correctly classified images (2013).[209] Goertzel hypothesized that these behaviors are due to limitations in their internal representations and that these limitations would inhibit integration into heterogeneous multi-component artificial general intelligence (AGI) architectures.[207] These issues may possibly be addressed by deep learning architectures that internally form states homologous to image-grammar[210] decompositions of observed entities and events.[207] Learning a grammar (visual or linguistic) from training data would be equivalent to restricting the system to commonsense reasoning that operates on concepts in terms of grammatical production rules and is a basic goal of both human language acquisition[211] and artificial intelligence (AI).[212]

Cyber threat
As deep learning moves from the lab into the world, research and experience show that artificial neural networks are vulnerable to hacks and deception.[213] By identifying patterns that these systems use to function, attackers can modify inputs to ANNs in such a way that the ANN finds a match that human observers would not recognize. For example, an attacker can make subtle changes to an image such that the ANN finds a match even though the image looks to a human nothing like the search target. Such manipulation is termed an “adversarial attack.”[214]

In 2016 researchers used one ANN to doctor images in trial and error fashion, identify another's focal points and thereby generate images that deceived it. The modified images looked no different to human eyes. Another group showed that printouts of doctored images then photographed successfully tricked an image classification system.[215] One defense is reverse image search, in which a possible fake image is submitted to a site such as TinEye that can then find other instances of it. A refinement is to search using only parts of the image, to identify images from which that piece may have been taken.[216]

Another group showed that certain psychedelic spectacles could fool a facial recognition system into thinking ordinary people were celebrities, potentially allowing one person to impersonate another. In 2017 researchers added stickers to stop signs and caused an ANN to misclassify them.[215]

ANNs can however be further trained to detect attempts at deception, potentially leading attackers and defenders into an arms race similar to the kind that already defines the malware defense industry. ANNs have been trained to defeat ANN-based anti-malware software by repeatedly attacking a defense with malware that was continually altered by a genetic algorithm until it tricked the anti-malware while retaining its ability to damage the target.[215]

In 2016, another group demonstrated that certain sounds could make the Google Now voice command system open a particular web address, and hypothesized that this could "serve as a stepping stone for further attacks (e.g., opening a web page hosting drive-by malware)."[215]

In “data poisoning,” false data is continually smuggled into a machine learning system's training set to prevent it from achieving mastery.

Reliance on human microwork

Most Deep Learning systems rely on training and verification data that is generated and/or annotated by humans. It has been argued in media philosophy that not only low-paid clickwork (e.g. on Amazon Mechanical Turk) is regularly deployed for this purpose, but also implicit forms of human microwork that are often not recognized as such.[217] The philosopher Rainer Mühlhoff distinguishes five types of "machinic capture" of human microwork to generate training data: (1) gamification (the embedding of annotation or computation tasks in the flow of a game), (2) "trapping and tracking" (e.g. CAPTCHAs for image recognition or click-tracking on Google search results pages), (3) exploitation of social motivations (e.g. tagging faces on Facebook to obtain labeled facial images), (4) information mining (e.g. by leveraging quantified-self devices such as activity trackers) and (5) clickwork.[217]

Mühlhoff argues that in most commercial end-user applications of Deep Learning such as Facebook's face recognition system, the need for training data does not stop once an ANN is trained. Rather, there is a continued demand for human-generated verification data to constantly calibrate and update the ANN. For this purpose Facebook introduced the feature that once a user is automatically recognized in an image, they receive a notification. They can choose whether of not they like to be publicly labeled on the image, or tell Facebook that it is not them in the picture.[218] This user interface is a mechanism to generate "a constant stream of verification data"[217] to further train the network in real-time. As Mühlhoff argues, involvement of human users to generate training and verification data is so typical for most commercial end-user applications of Deep Learning that such systems may be referred to as "human-aided artificial intelligence".

See also
Applications of artificial intelligence
Comparison of deep learning software
Compressed sensing
Differentiable programming
Echo state network
List of artificial intelligence projects
Liquid state machine
List of datasets for machine learning research
Reservoir computing
Scale space and deep learning
Sparse coding
"""

In [17]:
# remove [#]

for i in range(300):
  Document = Document.replace("["+str(i)+"]", " ")

Document = Document.replace("\n", " ").replace("(", "").replace(")", "").replace(",", "").replace(".", "")
print(str(len(Document)) + "\n")
print(Document)

46537

Deep learning also known as deep structured learning is part of a broader family of machine learning methods based on artificial neural networks with representation learning Learning can be supervised semi-supervised or unsupervised   Deep-learning architectures such as deep neural networks deep belief networks deep reinforcement learning recurrent neural networks and convolutional neural networks have been applied to fields including computer vision speech recognition natural language processing machine translation bioinformatics drug design medical image analysis climate science material inspection and board game programs where they have produced results comparable to and in some cases surpassing human expert performance     Artificial neural networks ANNs were inspired by information processing and distributed communication nodes in biological systems ANNs have various differences from biological brains Specifically artificial neural networks tend to be static and symbolic wh

In [6]:
words = Document.split(" ") # Split the Document into a list of words
words = [word for word in words if len(word) > 1] # Remove potential formatting issues with newlines (copy and pasted the string value for the Document)

passage_count = int(len(words) / 100) # How many 100-word passages are in this Document?
print("Passage count = " + str(passage_count + 1)) # add the remaining words into a passage of length < 100 at the end of the loop

passages = []
for i in range(0, len(words)-1, 100):
  passages.append(' '.join(words[i:i+100])) # join each 100 word chunk into a Passage string

# ^ We will split this again later to source the n-grams, but nice to visualize the passages as a string

passages.append(' '.join(words[i:])) # i will be 7200 or something at this point, so it chunks the last 97 words

print(passages[0]) # see some passages
print(passages[1])
print(passages[2])
# ...

Passage count = 68
Deep learning also known as deep structured learning is part of broader family of machine learning methods based on artificial neural networks with representation learning Learning can be supervised semi-supervised or unsupervised Deep-learning architectures such as deep neural networks deep belief networks deep reinforcement learning recurrent neural networks and convolutional neural networks have been applied to fields including computer vision speech recognition natural language processing machine translation bioinformatics drug design medical image analysis climate science material inspection and board game programs where they have produced results comparable to and in some cases surpassing human expert performance Artificial neural networks ANNs
were inspired by information processing and distributed communication nodes in biological systems ANNs have various differences from biological brains Specifically artificial neural networks tend to be static and symboli

In [7]:
# Remove last passage (Will fix later on to include this)
del(passages[67])

In [8]:
passage_n_grams = {} # Key, Value | Key = Passage Index, Value = n-gram list
for idx in range(len(passages)): # Loop through passages
  n_grams = [] # init new n_gram list
  passage_words = passages[idx].split(" ") # back to a list of words
  passage_words = [word for word in passage_words if len(word) > 1] # weird issue I was having with .split(" ")
  for jdx in range(0, 100): # start at each word to sample an n-gram window from left to right
    if jdx > 90: # avoid indexing errors > 100
      for n_gram_size in range(3, 100-jdx): # so if it is at 97 it will just get 1, 2, and 3 length n-grams
        new_n_gram = ' '.join(passage_words[jdx:jdx+n_gram_size]) # construct n-gram from list of words
        n_grams.append(new_n_gram) # add to n_gram
    else: # no worries with out of bounds indexing
      for n_gram_size in range(3, 10):
        new_n_gram = ' '.join(passage_words[jdx:jdx+n_gram_size])
        n_grams.append(new_n_gram)
  passage_n_grams[idx] = n_grams # set the Key = Passage Index, Value = n-gram list

print(passage_n_grams[0][:5]) # Visaulize a Sample

['Deep learning also', 'Deep learning also known', 'Deep learning also known as', 'Deep learning also known as deep', 'Deep learning also known as deep structured']


In [9]:
import time # Document how long this takes
start = time.time() # Start the stopwatch!
n_gram_match_dict = {} 
# This is going to be a Dictionary of Dictionaries,
# Outer Key = Passage Index | Inner Key = n_gram index | Inner Value = List of Matching Passage Indexes
# i.e.
'''
0: { # The first passage
   0: [1,2,5], # The first n-gram has matches in passages 2, 3, and 6
   1: [30, 55], # The second n-gram has matches in passages 31 and 55
   ...
},
1: { # The second passage
  8: [23, 25, 27], # The 9th n-gram of the second passage has 3 matches
  ...
}

Note we do not need to index passages that do not have any matches, because they are not recurring spans
'''
for idx in range(len(passages)): # loop through passages
  passage_match_dict = {} # Inner Dictionary of matches for this passage
  for jdx in range(len(passage_n_grams[idx])): # loop through each span in the passage
    matches = [] # init match list for this n-gramm
    for zdx in range(len(passages)): # loop through passages
      if zdx != idx: # Don't check for matches in the same passage!
        n_gram_set = set(passage_n_grams[zdx]) # convert n-grams to set for faster check
        if passage_n_grams[idx][jdx] in n_gram_set: # Is this n-gram in the other passage?
          matches.append(zdx) # If yes, add to match list!
      if (len(matches) > 0): # No need to add the match list if there are no matches
        passage_match_dict[jdx] = matches
  n_gram_match_dict[idx] = passage_match_dict # save this dictionary

print("Ran in: " + str(int(time.time() - start)) + " seconds.") # Stop the stopwatch!
n_gram_match_dict[0][14] # matches for the n-gram "also known as"

Ran in: 90 seconds.


[11]

In [14]:
length_span_dict = {} # Key = Length, Value = List of Spans

for idx in range(len(passages)): # loop through each of the passages
  for j in n_gram_match_dict[idx].keys(): # loop through each n-gram with a match in the dict
    key = len(passage_n_grams[idx][j].split(" ")) # counting number of words rather than characters
    if key in length_span_dict.keys(): # add n-gram to dict
      length_span_dict[key].append(passage_n_grams[idx][j])
    else: # init n-gram dict
      length_span_dict[key] = [passage_n_grams[idx][j]]

sorted(length_span_dict.items(), key = lambda item: item[0], reverse=True) # sort with longest first

[(9,
  ['and toxic effects of environmental chemicals in nutrients household',
   'toxic effects of environmental chemicals in nutrients household products',
   'effects of environmental chemicals in nutrients household products and',
   'of environmental chemicals in nutrients household products and drugs',
   'and toxic effects of environmental chemicals in nutrients household',
   'toxic effects of environmental chemicals in nutrients household products',
   'effects of environmental chemicals in nutrients household products and',
   'of environmental chemicals in nutrients household products and drugs']),
 (8,
  ['competitive with traditional speech recognizers on certain tasks',
   'and toxic effects of environmental chemicals in nutrients',
   'toxic effects of environmental chemicals in nutrients household',
   'effects of environmental chemicals in nutrients household products',
   'of environmental chemicals in nutrients household products and',
   'environmental chemicals in 